# Chatbot for game recommandation

In [1]:
!pip install langchain
!pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.9/351.9 kB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.7/431.7 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 86.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions

In [8]:
!pip uninstall openai langchain langchain_openai -y
!pip install openai langchain langchain_openai --upgrade

Found existing installation: openai 1.68.2
Uninstalling openai-1.68.2:
  Successfully uninstalled openai-1.68.2
Found existing installation: langchain 0.3.21
Uninstalling langchain-0.3.21:
  Successfully uninstalled langchain-0.3.21
Found existing installation: langchain-openai 0.3.10
Uninstalling langchain-openai-0.3.10:
  Successfully uninstalled langchain-openai-0.3.10
  Using cached openai-1.68.2-py3-none-any.whl.metadata (25 kB)
  Using cached langchain-0.3.21-py3-none-any.whl.metadata (7.8 kB)
  Using cached langchain_openai-0.3.10-py3-none-any.whl.metadata (2.3 kB)
Using cached openai-1.68.2-py3-none-any.whl (606 kB)
Using cached langchain-0.3.21-py3-none-any.whl (1.0 MB)
Using cached langchain_openai-0.3.10-py3-none-any.whl (61 kB)


In [5]:
#!pip show typing_extensions

In [2]:
import json
import numpy as np
from scipy.spatial.distance import cosine
from sentence_transformers import SentenceTransformer
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

### Embedding

In [3]:
from sentence_transformers import SentenceTransformer
import json

embedding_model = SentenceTransformer('all-MiniLM-L6-v2', cache_folder="./cache")


In [42]:

# Charger les jeux IGDB
#with open("igdb_games.json", "r") as f:
    #games_data = json.load(f)

#for game in games_data:
 #   game["embedding"] = embedding_model.encode(game.get("summary", "No description")).tolist()  

# Sauvegarde en JSON
#with open("igdb_games_embedding.json", "w") as f:
   # json.dump(games_data, f, indent=4)  

#print("Base de données des jeux enregistrée avec embeddings !")

### Sentiment analysis

In [4]:
# Load steam reviews
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords

steam_reviews = pd.read_csv("./steam_reviews.csv", names=['id', 'app_id', 'content', 'author_id', 'sentiment'], header=0)

In [5]:
# Preprocess the reviews
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
def clean_text(text):
    text = str(text)
    # remove html tags
    text = re.sub(r'<.*?>', '', text)

    # remove @mentions
    text = re.sub(r'@[A-Za-z0-9]+', '', text)

    # remove urls
    text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)

    # keep only alphanumeric characters
    text = re.sub(r'[^a-zA-Z]', ' ', text)

    return text

stop_words = set(stopwords.words('english'))
def remove_stopwords(tokens):
    return [word for word in tokens if word.lower() not in stop_words]

def remove_special_chars(tokens):
    return [word for word in tokens if word.isalnum()]

lemmatizer = WordNetLemmatizer()
def lemmatize(tokens):
    return [lemmatizer.lemmatize(word) for word in tokens]

def preprocess(tokens):
    # return lemmatize(remove_special_chars(remove_stopwords(tokens)))
    return lemmatize(remove_special_chars(tokens))

steam_reviews = steam_reviews[['content', 'sentiment']]
steam_reviews['content'] = steam_reviews['content'].apply(clean_text)
steam_reviews['sentiment'] = steam_reviews['sentiment'].apply(lambda x: str(x).lower())
steam_reviews.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,content,sentiment
0,At least its a counter strike,negative
1,Uh So far my playthrough has not been great...,negative
2,Better mechanics than cs,negative
3,buggy mess and NOT fun to play at all,negative
4,Whoever came up with this is gonna fucking ge...,negative


In [6]:
# Use tfidf to vectorize the reviews
from sklearn.feature_extraction.text import TfidfVectorizer

df = steam_reviews

tfidf = TfidfVectorizer(strip_accents=None, 
                        lowercase=False,
                        preprocessor=None)

X = tfidf.fit_transform(df['content'])
X.shape # (n_samples, n_features) = (201151, 97755)

(201151, 97755)

In [7]:
# Train the logistic regression model
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

y = df['sentiment'] # target variable
X_train, X_test, y_train, y_test = train_test_split(X,y)

lr = LogisticRegression(solver='liblinear')
lr.fit(X_train,y_train) # fit the model
preds = lr.predict(X_test) # make predictions

from sklearn.metrics import accuracy_score
accuracy_score(preds,y_test)

0.8405981546293351

In [8]:
# Predict the sentiment of a review
def predict_sentiment(review):
    review = clean_text(review)
    review = tfidf.transform([review])
    sentiment = lr.predict(review)[0]
    proba = lr.predict_proba(review)[0][0 if sentiment == 'negative' else 1]
    return sentiment, proba

predict_sentiment("I love game about war")

('positive', 0.9506846551103045)

### Information Extraction

In [9]:
!pip install fuzzywuzzy
!python -m spacy download en_core_web_md

2025-03-26 08:26:48.007187: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-26 08:26:48.007267: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-26 08:26:48.008896: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-26 08:26:48.017417: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-26 08:26:49.240573: W tensorflow/compiler/tf2

In [11]:
import spacy
from spacy.matcher import PhraseMatcher
#python -m spacy download en_core_web_md
from fuzzywuzzy import process  # Pour la recherche approximative
import random as random
# Charger le fichier JSON
with open("igdb_games_embedding.json", "r", encoding="utf-8") as file:
    games_data = json.load(file)
    
sample_games = random.sample(games_data, 1000)

# Initialiser les listes
games_db = []
categories_db = set()  # Utiliser un set pour éviter les doublons

# Parcourir les jeux dans le JSON
for game in sample_games:
    if "name" in game:
        games_db.append(game["name"])  # Ajouter le nom du jeu
    
    if "genres" in game and isinstance(game["genres"], list):
        for genre in game["genres"]:
            if "name" in genre:  # Vérifier si le nom du genre existe
                categories_db.add(genre["name"])

# Convertir en liste finale
categories_db = list(categories_db)

In [59]:

# Afficher le résultat
#print("Jeux détectés:", games_db[:10])
#print("Catégories détectées:", categories_db[:10])

In [12]:
# Charger le modèle spaCy
nlp = spacy.load("en_core_web_md")  # ou "fr_core_news_md" pour le français

# Initialisation du PhraseMatcher
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")

game_to_use=games_db
# Ajouter les noms de jeux et catégories au matcher
patterns_games = [nlp(game) for game in game_to_use]
patterns_categories = [nlp(cat) for cat in categories_db]

matcher.add("GAME", patterns_games)
matcher.add("CATEGORY", patterns_categories)

def fuzzy_match_game(text, threshold=80):
    """
    Recherche un jeu dans la base avec tolérance aux fautes de frappe.
    Renvoie le jeu correspondant si la similarité est supérieure au seuil.
    """
    best_match, score = process.extractOne(text, game_to_use) if text else (None, 0)
    return best_match if score >= threshold else None

def extract_games_and_categories(text):
    """
    Détecte les jeux et catégories dans un texte en combinant PhraseMatcher et fuzzy matching.
    """
    doc = nlp(text)  # Tokenisation
    matches = matcher(doc)  # Recherche des correspondances exactes
    
    extracted_info = {"games": set(), "categories": set()}
    matched_texts = set()  # Pour éviter les doublons

    # Recherche des jeux et catégories exacts avec PhraseMatcher
    for match_id, start, end in matches:
        label = nlp.vocab.strings[match_id]
        entity = doc[start:end].text
        
        if label == "GAME":
            extracted_info["games"].add(entity)
        elif label == "CATEGORY":
            extracted_info["categories"].add(entity)
        
        matched_texts.add(entity.lower())

    # 🔍 Recherche des jeux approximatifs avec filtre
    for token in doc:
        if (token.text.lower() not in matched_texts  # Éviter les doublons
            and not token.is_punct  # Ignorer la ponctuation
            and not token.is_stop  # Ignorer les mots vides ("le", "et", "de", etc.)
            and len(token.text) > 2):  # Ignorer les mots trop courts
        
            fuzzy_game = fuzzy_match_game(token.text)
            if fuzzy_game:
                extracted_info["games"].add(fuzzy_game)
    
    return extracted_info


In [13]:
#text = "J'adore Elden Ring, c'est un excellent RPG. Je cherche un jeu d'action qui ressemble à Hades. J'ai aussi aimé Dark Sols."

#result = extract_games_and_categories(text)
print(result)

NameError: name 'result' is not defined

### Function for recommendations

In [19]:
import numpy as np
from scipy.spatial.distance import cosine
import random

# Fonction de recommandation basée sur la similarité
def recommend_game(user_query):
    #extract infos
    extract = extract_games_and_categories(user_query)
    # Récupérer les descriptions des jeux extraits
    game_descriptions = {}
    for game_name in extract['games']:
        for game in sample_games:
            if game['name'] == game_name:
                game_descriptions[game_name] = "I want an adventure game"


    # Sentiment analysis
    sentiment, proba = predict_sentiment(user_query)

    recommand_similar_game = True # default
    if sentiment=='negative' and proba > 0.70:
        recommand_similar_game = False
        
    if game_descriptions:
        description_query = " ".join(game_descriptions.values())  # Combine toutes les descriptions
        user_query = description_query
    query_embedding = embedding_model.encode(user_query)
    similarities = [
        (game["name"], 1 - cosine(query_embedding, game["embedding"]), game.get("rating", 0))
        for game in sample_games if "embedding" in game
    ]

    if recommand_similar_game:
        similarities.sort(key=lambda x:(x[1], x[2]), reverse=True)
    else:
        similarities.sort(key=lambda x: (x[1], x[2]), reverse=False)

    return similarities[:3]

# Test
#user_input = "I don't love war game"
#print("Jeux recommandés :", recommend_game(user_input))


### LLM

In [20]:
class ChatSession:
    def __init__(self, memsize, llm):
        self.llm = llm
        self.memsize = memsize
        self.history = []
        self.llmID = "ASSISTANT:"
        self.userID = "USER:"
                
    def summarizeStrategy(self):
        summaryPrompt = "Résume la conversation suivante en gardant un maximum d'informations fournies par " + self.userID + " :\n"
        lastMessage, lastMessageLength = self.history[-1]
        history = ""
        for m, l in self.history[:-1]:
            history += m + "\n"
        summary = self.llm.invoke([HumanMessage(content=summaryPrompt + "\n" + history)]).content
        summaryLength = self.estimateLength(summary)
        if summaryLength + lastMessageLength > self.memsize:
            self.fifoStrategy()
            return
        self.history = [(summary, summaryLength), (lastMessage, lastMessageLength)]
    
    def fifoStrategy(self):
        while self.historyLength() > self.memsize:
            self.history = self.history[1:]

    def historyLength(self):
        return sum(length for _, length in self.history)

    def estimateLength(self, text):
        return self.llm.get_num_tokens(text)

    def addToHistory(self, prompt):
        self.history.append((prompt, self.estimateLength(prompt)))
        if self.historyLength() > self.memsize:
            self.summarizeStrategy()

    def chat(self, prompt):
        self.addToHistory(self.userID + " " + prompt)

        recommendations = recommend_game(prompt)
        games_list = ", ".join([game[0] for game in recommendations])

        chat_prompt = f"""
        L'utilisateur cherche un jeu correspondant à : "{prompt}".
        Voici les jeux recommandés : {games_list}.
        Formule une réponse naturelle et engageante.
        """

        fullPrompt = 'Historique de conversation :\n'
        for msg, _ in self.history:
            fullPrompt += msg + "\n"

        fullPrompt += "\n" + chat_prompt

        resp = self.llm.invoke([HumanMessage(content=fullPrompt)])
        content = resp.content
        if content.startswith(self.llmID + ": "):
            content = content[len(self.llmID + ": "):]

        self.addToHistory(self.llmID + " " + content)
        return content

In [21]:
 llm = ChatOpenAI(
        openai_api_key="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",
        openai_api_base="https://openrouter.ai/api/v1",
        model_name="mistralai/mistral-small-3.1-24b-instruct:free",
    )

In [22]:
CONTEXT_LENGTH = 500  
session = ChatSession(CONTEXT_LENGTH, llm)

### Use Of chatbot

In [23]:
print("\n Welcome to the video game recommendation chatbot with memory!")
print("\n Ask me a question (e.g.: “I want an RPG with a good scenario”).")
print("\n Type “exit” to quit.\n")

while True:
    user_input = input("👤 Vous : ")
    if user_input.lower() == "exit":
        print("À bientôt !")
        break
    
    response = session.chat(user_input)
    print(f"🤖 Bot : {response}\n")



 Welcome to the video game recommendation chatbot with memory!

 Ask me a question (e.g.: “I want an RPG with a good scenario”).

 Type “exit” to quit.



👤 Vous :  i want to have an adventure game


🤖 Bot : Bien sûr ! Voici quelques suggestions qui devraient te plaire si tu cherches un jeu d'aventure :

1. **The Unlikely Prometheus** : C'est un jeu indépendant avec une histoire captivante et des mécaniques de gameplay intéressantes. Parfait pour les amateurs de mystères et de découvertes.

2. **NetGame Adventure** : Si tu aimes les jeux en ligne avec une communauté active, celui-ci pourrait être fait pour toi. Il offre une expérience d'aventure en réseau, plein de défis et de surprises.

3. **Trance** : Pour ceux qui cherchent une expérience immersive et psychédélique, Trance est un excellent choix. Ce jeu explore des thèmes de réalité augmentée et de mystère, tout en t'offrant des combats et des puzzles stimulants.

Chacun de ces jeux a son propre style unique, donc tu devrais trouver quelque chose qui te convient parfaitement. Bonne aventure ! 🎮✨



👤 Vous :  i want to have a killer game


🤖 Bot : D'accord, si tu veux un jeu qui soit véritablement captivant et intense, voici quelques options parmi celles mentionnées qui pourraient te plaire :

1. **The Unlikely Prometheus** : Ce jeu est parfait si tu aimes les histoires riches en mystère et en rebondissements. C'est un véritablequête pleine de défis et de surprises qui te tiendra en haleine du début à la fin.

2. **NetGame Adventure** : Pour les amateurs de compétition et d'interaction en ligne, c'est le jeu idéal. Avec une communauté active, tu seras sûr de trouver des adversaires redoutables et des défis stimulants.

3. **Trance** : Ce jeu est parfait si tu cherches une expérience vraiment immersive et intense. Avec ses thèmes de réalité augmentée et ses puzzles complexes, Trance promet une aventure captivante et psychédélique.

Chacun de ces jeux a des éléments qui peuvent rendre ton expérience de jeu vraiment "killer". J'espère que tu trouveras celui qui correspond le mieux à tes attentes ! Bonne aventure et amuse-to

...

🤖 Bot : Super ! Si tu cherches un jeu d'aventure, tu as de superbes options à considérer. Chacun de ces jeux a ses propres atouts et peut te plonger dans des univers différents, voici un rappel :

1. **The Unlikely Prometheus** : Parfait pour ceux qui aiment les mystères et les découvertes. C'est une expérience indépendante avec une histoire captivante et des mécaniques de gameplay intéressantes.

2. **NetGame Adventure** : Idéal si tu préfères les expériences en ligne et interactives. Avec une communauté active, tu pourras relever des défis et t'immerger dans une aventure collaborative pleine de surprises.

3. **Trance** : Pour une expérience vraiment immersive et psychédélique, ce jeu explore des thèmes de réalité augmentée avec des puzzles et des combats stimulants. C'est une véritable plongée dans l'inconnu!

Chacun de ces jeux t'offrira une aventure inoubliable selon tes préférences. Bonne aventure et amuse-toi bien avec celui que tu choisiras! 🎮✨



👤 Vous :  I like cars and I want a game with


🤖 Bot : D'accord, j'ai noté que tu aimes les voitures et que tu cherches un jeu en rapport avec ça. Malheureusement, les jeux que j'ai mentionnés avant ne sont pas spécifiquement axés sur les voitures. Mais je peux te suggérer quelques alternatives qui pourraient bien te plaire :

1. **Forza Horizon 5** : Ce jeu te permet de conduire dans des environnements magnifiques et de participer à des courses époustouflantes. C'est parfait pour les amateurs de voitures et d'aventure.

2. **Gran Turismo Sport** : Si tu préfères quelque chose de plus réaliste, ce jeu offre une expérience de conduite très immersion avec des graphismes incroyables et des courses compétitives.

3. **Need for Speed Heat** : Pour une aventure plus arcade et des défis palpitants, ce jeu te propose des courses nocturnes illégales et des poursuites avec la police.

Chacun de ces jeux tiendra compte de ta passion pour les voitures et te proposera une aventure unique et passionnante. Bonne route et amuse-toi bien ! 🏎️🎮



👤 Vous :  exit


À bientôt !
